# Classical Forecasting Methods: Vector Autoregression (VAR)


**Approximate Learning Time**: Up to 2 hours

--- 

## Vector Autoregression (VAR) 


As the name suggests, VAR models perform auto-regression on vectors rather than scalars, as in univariate time series. VAR models follow a simple mathematical formulation and are trained using Maximum Likelihood Estimation (MLE) principles. They differ from VARMA models, which involve error terms and are optimized using iterative MLE. Plese refer to this [YouTube video]((https://www.youtube.com/watch?v=0-FKPJ5KxSo)) for a quick and simple introduction to VAR models.

For this tutorial, we will focus on VAR models, but you are welcome to explore VARMA models using the `VARMAX` module. Refer to the [documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.varmax.VARMAX.html#statsmodels.tsa.statespace.varmax.VARMAX) for more details. 

--- 

Let's load the log daily returns of exchange rates, and split the data into train, validation, and test subsets!



In [ ]:
import pathlib
import numpy as np
import pandas as pd
import itertools
from tqdm.notebook import tqdm
from termcolor import colored

import sys; sys.path.append("../")
import utils

from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

# To avoid flooding of the screen with convergence warnings during hyperparameter tuning
import warnings
warnings.filterwarnings("ignore")

## WARNING: To compare different models on the same horizon, keep this same across the notebooks
FORECASTING_HORIZON = [4, 8, 12] # weeks 
MAX_FORECASTING_HORIZON = max(FORECASTING_HORIZON)

SEQUENCE_LENGTH = 2 * MAX_FORECASTING_HORIZON
PREDICTION_LENGTH = MAX_FORECASTING_HORIZON

DIRECTORY_PATH_TO_SAVE_RESULTS = pathlib.Path('../results/DIY/').resolve()
MODEL_NAME = "VAR"

RESULTS_DIRECTORY = DIRECTORY_PATH_TO_SAVE_RESULTS / MODEL_NAME
if RESULTS_DIRECTORY.exists():
    print(colored(f'Directory {str(RESULTS_DIRECTORY)} already exists.'
           '\nThis notebook will overwrite results in the same directory.'
           '\nYou can also create a new directory if you want to keep this directory untouched.'
           ' Just change the `MODEL_NAME` in this notebook.\n', "red" ))
else:
    RESULTS_DIRECTORY.mkdir(parents=True)

data, transformed_data = utils.load_tutotrial_data(dataset='exchange_rate', log_transform=True)
data = transformed_data 

train_val_data = data.iloc[:-MAX_FORECASTING_HORIZON]
train_data, val_data = train_val_data.iloc[:-MAX_FORECASTING_HORIZON], train_val_data.iloc[-MAX_FORECASTING_HORIZON:]
test_data = data.iloc[-MAX_FORECASTING_HORIZON:]

print(f"Number of steps in training data: {len(train_data)}\nNumber of steps in validation data: {len(val_data)}\nNumber of steps in test data: {len(test_data)}")

%load_ext autoreload
%autoreload 2

---

## VAR / VARMAX Model

In this section, we will fit VAR and VARMAX models to the training data. 

We use the `VAR`([documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.var_model.VAR.html)) model from `statsmodels`. VAR models accept the parameter `max_lags` and chose the best autoregressive order accordingly. 

To find the best VARMA model, we will be using `VARMAX` ([documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.varmax.VARMAX.html)) from `statsmodels`. In contrast to VAR model, we need to specify the order. As a result, we will run a small hyperparameter search on the order. We will pick the one that has the best overall performance.

**Note**: We will chose the parameters based on the best mean mase across the time series.

Let's define a function that builds both the models, and call this function with different parameters to find the best parameters.


In [6]:
def train_and_forecast_VARMA(data, forecast_horizon, p=10, q=5):
    """
    Trains VARMA or VAR model depending on the parameters passed.

    Args:
        data: pandas DataFrame
        forecast_horizon: number fo steps to forecast
        p: autoregressive order
        q: moving average order; if q=0, p is assumed to be max_lags parameter of VAR Model
    """
    if q == 0:
        print(f"Training a VAR model; Selecting the best autoregressive order from p={{1, 2, ..., {p}}}")
        model = VAR(data).fit(maxlags=p, ic='aic') # note it will automatically select the best order for AR

        # make forecasts
        lag_order = model.k_ar # best chosen order
        print(f"Best p: {lag_order}")
        forecasted_values = model.forecast(data.values[-lag_order:], steps=forecast_horizon)
    else:
        print(f"Fitting VARMAX for parameters: p={p}\tq={q}\n")
        model = VARMAX(data, order=(p, q)).fit(maxiter=50, disp=False)
        forecasted_values = model.forecast(steps=MAX_FORECASTING_HORIZON) # make predictions 

    date_range = pd.date_range(start=data.index[-1], periods=forecast_horizon+1, freq=data.index.freq)
    date_range = date_range[1:] # we don't need the last index of train

    forecasted_df = pd.DataFrame(forecasted_values, index=date_range, columns=train_data.columns)
    
    return model, forecasted_df

Let's find the best VAR model!

In [ ]:
model_mase = {}

# train a VAR model
max_p = 10
model, forecasted_df = train_and_forecast_VARMA(train_data, forecast_horizon=MAX_FORECASTING_HORIZON, p=max_p, q=0)

# compute mase
all_mase_simple_var = utils.get_mase(forecasted_df, val_data, train_data)
model_mase[(max_p, )] = np.mean(list(all_mase_simple_var.values()))
print(f"MASE: {model_mase[(max_p, )]: 0.4f}")

Let's find the best VARMA model!

In [ ]:
# find the best VARMA model
p_values = [1, 2]
q_values = [1, 2]
pq_values = list(itertools.product(p_values, q_values))
print(f"Searching for best (p, q) among {len(pq_values)} combinations in VARMAX model")
for p,q in tqdm(pq_values):
    model, forecasted_df = train_and_forecast_VARMA(train_data, forecast_horizon=MAX_FORECASTING_HORIZON, p=p, q=q)

    # compute mase
    x = utils.get_mase(forecasted_df, val_data, train_data)
    model_mase[(p, q)] = np.mean(list(x.values()))
    print(f"MASE: {model_mase[(p, q)]: 0.4f}")

---

## Refit on Train-Val Subset & Forecast

To measure the model's performance on the test data, we will first retrain the model using the combined train-validation dataset. Then, we will compute the MASE metric on the test dataset to evaluate its performance.
Additionally, we will store the test predictions for later comparison with other forecasting methods.

We observe that the simple VAR model is better so we use that here.


In [ ]:
best_params = [key for key in model_mase.keys() if model_mase[key] == min(model_mase.values())][0]
p = best_params[0]
q = 0 if len(best_params) == 1 else best_params[1]
print(f"Best parameters: p={p} q={q}")
model, forecasted_df = train_and_forecast_VARMA(train_val_data, forecast_horizon=MAX_FORECASTING_HORIZON, p=p, q=q)

AUGMENTED_COL_NAMES = [f"{MODEL_NAME}_{col}_mean" for col in data.columns]
test_predictions_df = pd.DataFrame(forecasted_df.values, columns=AUGMENTED_COL_NAMES, index=test_data.index)
test_predictions_df.to_csv(f"{str(RESULTS_DIRECTORY)}/predictions.csv", index=True)
test_predictions_df.head()

---

## Evaluate

Let's compute the metrics by comparing the predictions with that of the target data. Note that we will have to rename the columns of the dataframe to match the expected column names by the function. 

In [ ]:
# compute MASE metrics
model_metrics, records = utils.get_mase_metrics(
    historical_data=train_val_data,
    test_predictions=test_predictions_df.rename(
            columns={x:x.split("_")[1] for x in test_predictions_df.columns
        }),
    target_data=test_data,
    forecasting_horizons=FORECASTING_HORIZON,
    columns=data.columns, 
    model_name=MODEL_NAME,
)

records = pd.DataFrame(records)

records.to_csv(f"{str(RESULTS_DIRECTORY)}/metrics.csv", index=False)
records[['col', 'horizon', 'mase']].pivot(index=['horizon'], columns='col')


---

## Compare Models

In [ ]:
utils.display_results(path=DIRECTORY_PATH_TO_SAVE_RESULTS, metric='mase')

---

## Plot Forecasts

In [ ]:
fig, axs = utils.plot_forecasts(
    historical_data=train_val_data,
    forecast_directory_path=DIRECTORY_PATH_TO_SAVE_RESULTS,
    target_data=test_data,
    columns=data.columns,
    n_history_to_plot=10, 
    forecasting_horizon=MAX_FORECASTING_HORIZON,
    dpi=200,
    plot_se=False
)

--- 

## Conclusions

We explored classical multivariate time series forecasting methods, specifically **VAR** (Vector Autoregression) and **VARMAX** (Vector Autoregressive Moving Average with Exogenous Variables), as generalizations of ARIMA for multivariate time series. We trained these models with various parameter configurations and evaluated the performance of the best model.

--- 

## Exercises

- Check the effect of hyperparameter `trend` on model performances above

- Apply a normalization procedure (e.g., **min-max scaling**) to the data, ensuring that only the training data is used for fitting the scaler. Perform the modeling process on the normalized data and, after generating the final model's predictions, invert the normalization to return the output to its original scale. See `sklearn.preprocessing.MinMaxScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html))

- Additionally, perform the modeling on the **raw data**, without applying any transformation (such as converting it into log daily returns), to compare results directly with the untransformed dataset.

---

## Next Steps

To learn about machine learning based approaches, check out the module 4 (XGBoost), module 5 (LSTM-based models), module 6 (Transformer-based models), or module 7 (LLM-based models).

---